In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

filepath  xmin  xmax  ymin  ymax
0    ./images\N1.xml  1093  1396   645   727
1  ./images\N100.xml   134   301   312   350
2  ./images\N101.xml    31   139   128   161
3  ./images\N102.xml   164   316   216   243
4  ./images\N103.xml   813  1067   665   724

In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./images\\N1.xml'

In [5]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images',filename_image)
    return filepath_image

In [6]:
getFilename(filename)

'./images\\N1.jpeg'

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path 

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N11.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N12.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N14.jpeg

#### verify image and output

In [8]:
file_path = image_path[0]
file_path 

'./images\\N1.jpeg'

In [9]:
img = cv2.imread(file_path)

cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# 1093	1396	645	727
cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Data Preprocessing

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [13]:
labels = df.iloc[:,1:].values

In [14]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # normalized output
    # -------------- append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [15]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [16]:
X.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [17]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

### Deep Learning Model

In [18]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [19]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,
                                     input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [29]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

Total params: 73,663,490
Trainable params: 19,326,754
Non-trainable params: 54,336,736
__________________________________________________________________________________________________


### model training

In [30]:
from tensorflow.keras.callbacks import TensorBoard

In [31]:
tfb = TensorBoard('object_detection')

In [32]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 [==============================] - 39s 2s/step - loss: 0.1061 - val_loss: 0.0768
Epoch 2/100
18/18 [==============================] - 28s 2s/step - loss: 0.0670 - val_loss: 0.0545
Epoch 3/100
18/18 [==============================] - 29s 2s/step - loss: 0.0506 - val_loss: 0.0517
Epoch 4/100
18/18 [==============================] - 29s 2s/step - loss: 0.0430 - val_loss: 0.0476
Epoch 5/100
18/18 [==============================] - 29s 2s/step - loss: 0.0398 - val_loss: 0.0452
Epoch 6/100
18/18 [==============================] - 30s 2s/step - loss: 0.0373 - val_loss: 0.0406
Epoch 7/100
18/18 [==============================] - 30s 2s/step - loss: 0.0363 - val_loss: 0.0408
Epoch 8/100
18/18 [==============================] - 30s 2s/step - loss: 0.0366 - val_loss: 0.0427
Epoch 9/100
18/18 [==============================] - 30s 2s/step - loss: 0.0357 - val_loss: 0.0388
Epoch 10/100
18/18 [==============================] - 31s 2s/step - loss: 0.0347 - val_loss: 0.0406
Epoch 11/

In [33]:
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=200,
                    validation_data=(x_test,y_test),callbacks=[tfb],initial_epoch=100)

Epoch 101/200
18/18 [==============================] - 46s 3s/step - loss: 7.3248e-04 - val_loss: 0.0066
Epoch 102/200
18/18 [==============================] - 45s 3s/step - loss: 7.1495e-04 - val_loss: 0.0069
Epoch 103/200
18/18 [==============================] - 45s 3s/step - loss: 8.2310e-04 - val_loss: 0.0070
Epoch 104/200
18/18 [==============================] - 46s 3s/step - loss: 9.8306e-04 - val_loss: 0.0073
Epoch 105/200
18/18 [==============================] - 46s 3s/step - loss: 0.0012 - val_loss: 0.0069
Epoch 106/200
18/18 [==============================] - 46s 3s/step - loss: 0.0013 - val_loss: 0.0071
Epoch 107/200
18/18 [==============================] - 45s 3s/step - loss: 0.0012 - val_loss: 0.0063
Epoch 108/200
18/18 [==============================] - 46s 3s/step - loss: 0.0011 - val_loss: 0.0064
Epoch 109/200
18/18 [==============================] - 46s 3s/step - loss: 0.0012 - val_loss: 0.0072
Epoch 110/200
18/18 [==============================] - 46s 3s/step - loss: 

In [34]:
model.save('./models/object_detection.h5')